In [169]:
%load_ext autoreload
%autoreload 2

import util
import config


project_id = 119867
pool_id = 36425808

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [344]:
from concurrent.futures import ProcessPoolExecutor

In [345]:
pool = ProcessPoolExecutor(max_workers=8)

In [256]:
import pandas as pd
from collections import Counter

In [170]:
from tqdm.auto import tqdm

In [171]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch("http://localhost:9200", basic_auth=('elastic', 'changeme'))
es_client.info()

ObjectApiResponse({'name': '29617d95b9d1', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'axIx5HhRQY6iIouQ_OkZ1A', 'version': {'number': '8.5.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'c1310c45fc534583afe2c1c03046491efba2bba2', 'build_date': '2022-11-09T21:02:20.169855900Z', 'build_snapshot': False, 'lucene_version': '9.4.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [188]:
def save_hashes(attachment_id, doc):
    es_client.index(index="attachment", id=attachment_id, document=doc)

def find_duplicates(doc):
    keys = ['dhash', 'phash', 'md5']
    should = [{"term": {k: doc[k]}} for k in keys]
    query = {'bool': {'should': should}}

    response = es_client.search(query=query, index='attachment')
    return response['hits']

def has_duplicates(doc):
    response = find_duplicates(doc)
    return len(response['hits']) > 0

In [316]:
def get_available_suits(pool_id, last_id=None, status='SUBMITTED'):
    tasks_url = f'api/v1/assignments'

    params = {
        'status': status,
        'pool_id': pool_id,
        'sort': 'id',
        'limit': 100,
    }

    if last_id is not None:
        params['id_gt'] = last_id
    
    suits = []
    ids = set()

    while True:
        resp = util.toloka_get(tasks_url, params).json()

        items = resp['items']
        suits.extend(items)
        ids.update({i['id'] for i in items})

        print(f'added {len(items)}, now have {len(suits)} suits ({len(ids)} unique)')

        if resp['has_more'] == False:
            break

        params['id_gt'] = items[-1]['id']

    return suits, items[-1]['id']

In [317]:
all_suits, last_id = get_available_suits(pool_id, last_id='00022bd050--63777b7a82a8db7f5ddf6658')

added 100, now have 100 suits (100 unique)
added 100, now have 200 suits (200 unique)
added 100, now have 300 suits (300 unique)
added 100, now have 400 suits (400 unique)
added 100, now have 500 suits (500 unique)
added 100, now have 600 suits (600 unique)
added 100, now have 700 suits (700 unique)
added 100, now have 800 suits (800 unique)
added 100, now have 900 suits (900 unique)
added 100, now have 1000 suits (1000 unique)
added 100, now have 1100 suits (1100 unique)
added 100, now have 1200 suits (1200 unique)
added 100, now have 1300 suits (1300 unique)
added 100, now have 1400 suits (1400 unique)
added 100, now have 1500 suits (1500 unique)
added 100, now have 1600 suits (1600 unique)
added 100, now have 1700 suits (1700 unique)
added 100, now have 1800 suits (1800 unique)
added 100, now have 1900 suits (1900 unique)
added 100, now have 2000 suits (2000 unique)
added 100, now have 2100 suits (2100 unique)
added 100, now have 2200 suits (2200 unique)
added 100, now have 2300 sui

In [323]:
last_id

'00022bd050--6378039edb0e8f36d8eea3e5'

In [342]:
image_urls = []

for suite in all_suits:
    for sol in suite['solutions']:
        image_urls.append(sol['output_values']['image'])
        
len(image_urls)

4115

In [ ]:
image_hashes = util.map_progress(pool, image_urls, util.download_image_and_calc_hashes)

  0%|          | 0/4115 [00:00<?, ?it/s]

In [324]:
processed = set()
to_check = []

In [334]:
for suite in tqdm(all_suits):
    suite_id = suite['id']
    if suite_id in processed:
        print(f'aready processed {suite_id}')
        continue

    task = suite['tasks'][0]
    solution = suite['solutions'][0]

    output_values = solution['output_values']

    output_class = output_values['class']

    if output_class == 'other':
        util.reject(suite_id, 'wrong item type')
        processed.add(suite_id)
        continue

    image = output_values['image']
    hashes = image_hashes[image] #util.download_image_and_calc_hashes(image)
    
    if hashes is None:
        util.reject(suite_id, 'problems with image')
        processed.add(suite_id)
        continue

    if has_duplicates(hashes):
        util.reject(suite_id, 'duplicate')
        processed.add(suite_id)
        continue

    save_hashes(image, hashes)

    image_url = util.make_upload_thumbnail(pool_id, image)
    
    to_check.append({
        'image_url': image_url,
        'worker_id': suite['user_id'],
        'assignment_id': suite_id,
        'attachment_id': image
    })
    
    processed.add(suite_id)

  0%|          | 0/4115 [00:00<?, ?it/s]

aready processed 00022bd050--63777b7b8feaf02283166b1c
aready processed 00022bd050--63777b7d19c2d72a00af1d6c
aready processed 00022bd050--63777b7d7c35c13555fa66eb
aready processed 00022bd050--63777b7ed23d9327ecf8b024
aready processed 00022bd050--63777b81d23d9327ecf8b050
aready processed 00022bd050--63777b81d3a4cf610197168f
aready processed 00022bd050--63777b848408eb4b99a1b749
aready processed 00022bd050--63777b87c9e3e243ce8f47c1
aready processed 00022bd050--63777b87d23d9327ecf8b0a2
aready processed 00022bd050--63777b889376542ef78493ca
aready processed 00022bd050--63777b889717e270005c8b96
aready processed 00022bd050--63777b8b441f157a2a4e6545
aready processed 00022bd050--63777b8b8cae8c05c821c8a3
aready processed 00022bd050--63777b8c8d6def0568df03d1
aready processed 00022bd050--63777b8cee7d356a0872aede
aready processed 00022bd050--63777b8fdb0e8f36d8e9a149
aready processed 00022bd050--63777b911d6dd41f3b97ea32
aready processed 00022bd050--63777b93838d535336992c7f
aready processed 00022bd050-

saved a thumbnail to thumbnails/eu.76b66be8-9bc3-44c1-829a-8ca93bb94e60.jpg
downloading images/eu.bb922e9d-fa22-421b-9c1d-33d91ddac203...
saved a thumbnail to thumbnails/eu.bb922e9d-fa22-421b-9c1d-33d91ddac203.jpg
downloading images/eu.f79a549b-8c7e-48f2-8d9c-5d80a8f3b634...
saved a thumbnail to thumbnails/eu.f79a549b-8c7e-48f2-8d9c-5d80a8f3b634.jpg
00022bd050--63778317ebd2c7014fde4f62 REJECTED wrong item type
downloading images/eu.e07f9b09-5791-4354-9dde-68e78f7a6380...
saved a thumbnail to thumbnails/eu.e07f9b09-5791-4354-9dde-68e78f7a6380.jpg
downloading images/eu.569ff432-7626-443a-911b-3c19e1da135b...
saved a thumbnail to thumbnails/eu.569ff432-7626-443a-911b-3c19e1da135b.jpg
downloading images/eu.577584f3-f92d-407a-abd2-1d17a94ddcf1...
saved a thumbnail to thumbnails/eu.577584f3-f92d-407a-abd2-1d17a94ddcf1.jpg
downloading images/eu.598883e2-a30e-4892-8b30-31d51dcb39e5...
saved a thumbnail to thumbnails/eu.598883e2-a30e-4892-8b30-31d51dcb39e5.jpg
downloading images/eu.f2997ea2-9e9

saved a thumbnail to thumbnails/eu.c4d28a6d-c5b1-46a5-9ce3-9cf030633155.jpg
downloading images/eu.4f607ca9-085a-4045-b0c8-f910888f46d8...
saved a thumbnail to thumbnails/eu.4f607ca9-085a-4045-b0c8-f910888f46d8.jpg
downloading images/eu.b9475831-bc2f-46da-b924-922ea65f4eae...
saved a thumbnail to thumbnails/eu.b9475831-bc2f-46da-b924-922ea65f4eae.jpg
downloading images/eu.0cb8aa8e-7c33-4da7-a3a0-916be302a842...
saved a thumbnail to thumbnails/eu.0cb8aa8e-7c33-4da7-a3a0-916be302a842.jpg
00022bd050--6377dfec14cd5667369c9b8f REJECTED wrong item type
downloading images/eu.7c42090b-a963-4e55-be9a-f6e73b6e33e1...
saved a thumbnail to thumbnails/eu.7c42090b-a963-4e55-be9a-f6e73b6e33e1.jpg
downloading images/eu.f39e2d75-0a98-43b2-bfc1-fd32aaeef800...
saved a thumbnail to thumbnails/eu.f39e2d75-0a98-43b2-bfc1-fd32aaeef800.jpg
downloading images/eu.3d4637b7-0159-4e28-bea6-6db9907cc44e...
saved a thumbnail to thumbnails/eu.3d4637b7-0159-4e28-bea6-6db9907cc44e.jpg
downloading images/eu.01a9e569-0a6

saved a thumbnail to thumbnails/eu.115ef8d4-ca0e-4858-a5f6-bed524aa8b30.jpg
downloading images/eu.be25c34f-b7b0-4136-a0fa-65bc55481aa9...
saved a thumbnail to thumbnails/eu.be25c34f-b7b0-4136-a0fa-65bc55481aa9.jpg
00022bd050--6377e0cf9376542ef788c497 REJECTED wrong item type
downloading images/eu.013d99c7-9240-447e-8ae9-677c9b1b9b63...
saved a thumbnail to thumbnails/eu.013d99c7-9240-447e-8ae9-677c9b1b9b63.jpg
downloading images/eu.a16a13a7-b993-4521-a058-d16d6d79efe3...
saved a thumbnail to thumbnails/eu.a16a13a7-b993-4521-a058-d16d6d79efe3.jpg
00022bd050--6377e0d5d3a4cf61019af487 REJECTED wrong item type
downloading images/eu.4264f45e-4547-4908-aa1a-984aff9e16e6...
saved a thumbnail to thumbnails/eu.4264f45e-4547-4908-aa1a-984aff9e16e6.jpg
00022bd050--6377e0e2bd8f2378d352a354 REJECTED wrong item type
downloading images/eu.ae16bd41-e8a1-4332-948e-6ba9bc69b2fc...
saved a thumbnail to thumbnails/eu.ae16bd41-e8a1-4332-948e-6ba9bc69b2fc.jpg
downloading images/eu.31ef33ba-be8f-40cf-8c94-23

...

00022bd050--6377e8e4838d5353369f0275 REJECTED wrong item type
downloading images/eu.e79c2fe0-a060-4ada-b883-04fc3e49abb8...
saved a thumbnail to thumbnails/eu.e79c2fe0-a060-4ada-b883-04fc3e49abb8.jpg
downloading images/eu.e0e6f353-809f-4414-af89-1d53c36c6333...
saved a thumbnail to thumbnails/eu.e0e6f353-809f-4414-af89-1d53c36c6333.jpg
downloading images/eu.ae4f11a8-a84c-4a90-a959-a21d3c063cab...
saved a thumbnail to thumbnails/eu.ae4f11a8-a84c-4a90-a959-a21d3c063cab.jpg
downloading images/eu.0242385c-3575-481b-a11a-0024b5827fc2...
saved a thumbnail to thumbnails/eu.0242385c-3575-481b-a11a-0024b5827fc2.jpg
downloading images/eu.6164eb5d-a76d-4f55-bff0-e115d4cc67ae...
saved a thumbnail to thumbnails/eu.6164eb5d-a76d-4f55-bff0-e115d4cc67ae.jpg
downloading images/eu.84721b8f-cce2-4827-b317-ab9d4cbd543c...
saved a thumbnail to thumbnails/eu.84721b8f-cce2-4827-b317-ab9d4cbd543c.jpg
00022bd050--6377e8fd85bacb70b6763e4e REJECTED wrong item type
downloading images/eu.18a02465-7860-449f-875c-c6

saved a thumbnail to thumbnails/eu.1c822a33-f8d4-439f-8d49-4d1d96d3105b.jpg
downloading images/eu.9392ea91-e43e-43c8-bdca-2dc0a997c36f...
saved a thumbnail to thumbnails/eu.9392ea91-e43e-43c8-bdca-2dc0a997c36f.jpg
00022bd050--6377e9979717e2700060c8dd REJECTED wrong item type
downloading images/eu.f7ebdc97-2219-42a1-818d-317390000a56...
saved a thumbnail to thumbnails/eu.f7ebdc97-2219-42a1-818d-317390000a56.jpg
00022bd050--6377e999bdfa5861cdfd49e8 REJECTED wrong item type
downloading images/eu.918a479a-b31e-4083-aee1-a5a4cb0bfa94...
saved a thumbnail to thumbnails/eu.918a479a-b31e-4083-aee1-a5a4cb0bfa94.jpg
downloading images/eu.4c09bb48-daf9-41ef-975b-5ac8e1070eb7...
saved a thumbnail to thumbnails/eu.4c09bb48-daf9-41ef-975b-5ac8e1070eb7.jpg
downloading images/eu.1de6a96c-741b-4de0-a53d-5b6810d38f11...
saved a thumbnail to thumbnails/eu.1de6a96c-741b-4de0-a53d-5b6810d38f11.jpg
downloading images/eu.1f4d9082-7eab-420d-8da2-1ee8040f1eab...
saved a thumbnail to thumbnails/eu.1f4d9082-7eab

saved a thumbnail to thumbnails/eu.c442d48a-0683-4eb2-81a7-61febaa2aea2.jpg
downloading images/eu.f1614d2a-773f-4477-b1f2-33579b836864...
saved a thumbnail to thumbnails/eu.f1614d2a-773f-4477-b1f2-33579b836864.jpg
downloading images/eu.491bb79c-afff-4f08-9c6d-d0c0b999545c...
saved a thumbnail to thumbnails/eu.491bb79c-afff-4f08-9c6d-d0c0b999545c.jpg
downloading images/eu.1219be5a-e0ca-4a31-81a5-1d71f297eaf4...
saved a thumbnail to thumbnails/eu.1219be5a-e0ca-4a31-81a5-1d71f297eaf4.jpg
00022bd050--6377ea63b2c0c25adce2cf65 REJECTED wrong item type
downloading images/eu.4ea208bd-d97e-4fc3-bf65-942e815948a4...
saved a thumbnail to thumbnails/eu.4ea208bd-d97e-4fc3-bf65-942e815948a4.jpg
00022bd050--6377ea6f7adbfb533edaa852 REJECTED wrong item type
00022bd050--6377ea767adbfb533edaa897 REJECTED wrong item type
downloading images/eu.ad593437-80c5-4b62-866d-615a962b68b8...
saved a thumbnail to thumbnails/eu.ad593437-80c5-4b62-866d-615a962b68b8.jpg
downloading images/eu.31ae2365-a111-4223-b01c-b6

saved a thumbnail to thumbnails/eu.06d73286-b390-432b-b483-2abeffd32b69.jpg
downloading images/eu.b37614a5-a9d3-442f-ac48-09c82629350f...
saved a thumbnail to thumbnails/eu.b37614a5-a9d3-442f-ac48-09c82629350f.jpg
downloading images/eu.8bbf32c7-6262-4008-bc71-dbff8ee1455d...
saved a thumbnail to thumbnails/eu.8bbf32c7-6262-4008-bc71-dbff8ee1455d.jpg
downloading images/eu.507e32d4-bf5b-4f43-82e1-9e8c6aa5f94d...
saved a thumbnail to thumbnails/eu.507e32d4-bf5b-4f43-82e1-9e8c6aa5f94d.jpg
00022bd050--6377eb821455e654a9288ad9 REJECTED wrong item type
downloading images/eu.da6b5b0d-2857-418d-9bf3-1dd5f9153df8...
saved a thumbnail to thumbnails/eu.da6b5b0d-2857-418d-9bf3-1dd5f9153df8.jpg
downloading images/eu.f5dee481-643e-4237-8ee5-0ece642c6ed0...
saved a thumbnail to thumbnails/eu.f5dee481-643e-4237-8ee5-0ece642c6ed0.jpg
downloading images/eu.723b1aca-cbb4-490b-9c80-9adad2f0f31a...
saved a thumbnail to thumbnails/eu.723b1aca-cbb4-490b-9c80-9adad2f0f31a.jpg
downloading images/eu.0c4a1a25-380

KeyboardInterrupt: 

## Now let's upload this to Toloka!

In [ ]:
def make_batches(seq, n):
    result = []
    for i in range(0, len(seq), n):
        batch = seq[i:i+n]
        result.append(batch)
    return result

In [ ]:
pool_verify_id = 36426484
pool_upload_url = 'api/v1/task-suites?async_mode=false&open_pool=true'

In [ ]:
batches = make_batches(to_check, 10)

In [ ]:
len(batches)

In [ ]:
for batch in batches:
    tasks = [{'input_values': d} for d in batch]

    data = {
        'pool_id': pool_verify_id,
        'tasks': tasks,
        "overlap": 5,
    }

    resp = util.toloka_post(pool_upload_url, data)
    print(resp)
    print(resp.json())

## Accept/reject the original images

In [206]:
def get_available_suits(pool_id, last_id=None, status='SUBMITTED'):
    tasks_url = f'api/v1/assignments'

    params = {
        'status': status,
        'pool_id': pool_id,
        'sort': 'id',
        'limit': 100,
    }

    if last_id is not None:
        params['id_gt'] = last_id
    
    suits = []
    ids = set()

    while True:
        resp = util.toloka_get(tasks_url, params).json()

        items = resp['items']
        suits.extend(items)
        ids.update({i['id'] for i in items})

        print(f'added {len(items)}, now have {len(suits)} suits ({len(ids)} unique)')

        if resp['has_more'] == False:
            break

        params['id_gt'] = items[-1]['id']

    return suits, items[-1]['id']

In [207]:
pool_verify_id = 36426484


In [328]:
verify_suits[-1]

{'id': '00022bd2f4--6377947f8ab0c64deb6f30da',
 'task_suite_id': '00022bd2f4--6377934b8feaf0228317a212',
 'pool_id': '36426484',
 'user_id': '4ac1cce9ea5edcc9d3209fcd8ddedb2d',
 'status': 'ACCEPTED',
 'reward': 0.02,
 'tasks': [{'id': '0a4aed74-6e7a-45b2-bab4-e51d62e84a9c',
   'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.c437c89e-0017-405f-a6a4-333f98b9e2a8.jpg',
    'worker_id': '0310bfbc64effa398f2f202cbcceef84',
    'assignment_id': '00022bd050--63775de4b2c0c25adcda7cbe',
    'attachment_id': 'eu.c437c89e-0017-405f-a6a4-333f98b9e2a8'}},
  {'id': 'ede88170-eee2-46cc-9372-8199dc598b65',
   'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.567106c3-b4ea-4926-8bb5-5e74de339343.jpg',
    'worker_id': '12a408cf010c22553dced80ce6c66f2b',
    'assignment_id': '00022bd050--63775de4bdfa5861cdf6ce93',
    'attachment_id': 'eu.567106c3-b4ea-4926-8bb5-5e74de339343'}},
  {'id':

In [331]:
verify_suits[-1]['tasks'][-1]['id']

'f3e3c303-2fc1-4137-9296-176b69ba1b05'

In [332]:
verify_suits[-1]['id']

'00022bd2f4--6377947f8ab0c64deb6f30da'

In [208]:
verify_suits, last_id = get_available_suits(pool_verify_id, status='ACCEPTED')

added 100, now have 100 suits (100 unique)
added 100, now have 200 suits (200 unique)
added 100, now have 300 suits (300 unique)
added 100, now have 400 suits (400 unique)
added 100, now have 500 suits (500 unique)
added 100, now have 600 suits (600 unique)
added 100, now have 700 suits (700 unique)
added 100, now have 800 suits (800 unique)
added 14, now have 814 suits (814 unique)


In [235]:
data = []

for suite in verify_suits:
    for task, solution in zip(suite['tasks'], suite['solutions']):
        input_values = task['input_values']
        output_values = solution['output_values']
    
        row = input_values.copy()
        row.update(output_values)
        row['verifier'] = suite['user_id']
        data.append(row)

In [236]:
len(data)

9702

In [237]:
df = pd.DataFrame(data)

In [280]:
df.verdict.value_counts()

yes         9081
other        377
internet     244
Name: verdict, dtype: int64

In [238]:
gb = df.groupby('assignment_id')

In [274]:
all_results = []

for assignment_id, df_group in gb:
    classes = Counter(df_group['class'])
    verdicts = Counter(df_group['verdict'])
    
    most_common_verdict, _ = verdicts.most_common(1)[0]
    most_common_class, num_votes = classes.most_common(1)[0]

    agreement = num_votes / len(df_group)

    record = {
        'assignment_id': assignment_id,
        'agreement': agreement,
        'image': df_group.iloc[0].image_url,
        'most_common_class': most_common_class,
        'most_common_verdict': most_common_verdict,
        'verdicts': verdicts,
        'classes': classes,
    }
    
    all_results.append(record)

In [275]:
df_agreements = pd.DataFrame(all_results)

In [294]:
good = (df_agreements.most_common_verdict == 'yes') & \
    (df_agreements.most_common_class != 'other') & \
    (df_agreements.agreement >= 0.34)

In [295]:
sum(good) / len(good)

0.9160929432013769

In [297]:
df_agreements['final_verdict'] = 'ACCEPT'

In [299]:
df_agreements.loc[~good, 'final_verdict'] = 'REJECT'

In [302]:
df_agreements['comment'] = 'good job'

In [304]:
df_agreements.loc[~good, 'comment'] = "doesn't satisfy the requrements"

In [313]:
for row in tqdm(df_agreements.itertuples()):
    assignment_id = row.assignment_id
    verdict = row.final_verdict
    comment = row.comment
    
    if verdict == 'ACCEPT':
        util.accept(assignment_id)
    else:
        util.reject(assignment_id, comment)

0it [00:00, ?it/s]

reject 00022bbe55--63764ed07c35c13555f034b3: doesn't satisfy the requrements
00022bbe55--63764ed07c35c13555f034b3 REJECTED doesn't satisfy the requrements
00022bbe55--63764ed67c35c13555f03546 ACCEPTED good job
00022bbe55--63764eda441f157a2a3fb9cf ACCEPTED good job
00022bbe55--63764eda9376542ef779338a ACCEPTED good job
00022bbe55--63764eddc9e3e243ce841375 ACCEPTED good job
00022bbe55--63764eddee7d356a0865ee2e ACCEPTED good job
reject 00022bbe55--63764ee2d3c878690bc5bee6: doesn't satisfy the requrements
00022bbe55--63764ee2d3c878690bc5bee6 REJECTED doesn't satisfy the requrements
00022bbe55--63764ee35a4d4b776b0dee99 ACCEPTED good job
reject 00022bbe55--63764ee43903154c8badb2e8: doesn't satisfy the requrements
00022bbe55--63764ee43903154c8badb2e8 REJECTED doesn't satisfy the requrements
00022bbe55--63764ee4441f157a2a3fba6f ACCEPTED good job
reject 00022bbe55--63764ee77798532317b6ae8d: doesn't satisfy the requrements
00022bbe55--63764ee77798532317b6ae8d REJECTED doesn't satisfy the requrem

00022bd050--637695e256a5a54ced7318ef ACCEPTED good job
reject 00022bd050--637695e5ea9fec14e37b0a96: doesn't satisfy the requrements
00022bd050--637695e5ea9fec14e37b0a96 REJECTED doesn't satisfy the requrements
00022bd050--637695e856a5a54ced7319e3 ACCEPTED good job
00022bd050--637695ea8ab0c64deb658501 ACCEPTED good job
00022bd050--637695f06b2d840c16ea0882 ACCEPTED good job
00022bd050--637695f0ea9fec14e37b0c33 ACCEPTED good job
reject 00022bd050--637695f256a5a54ced731c92: doesn't satisfy the requrements
00022bd050--637695f256a5a54ced731c92 REJECTED doesn't satisfy the requrements
00022bd050--637695f39376542ef77d0b68 ACCEPTED good job
00022bd050--637695f58cae8c05c8193a6d ACCEPTED good job
00022bd050--637695fc56a5a54ced731e0e ACCEPTED good job
00022bd050--637695ffee7d356a0869c673 ACCEPTED good job
reject 00022bd050--63769600b1ad940367f1b57f: doesn't satisfy the requrements
00022bd050--63769600b1ad940367f1b57f REJECTED doesn't satisfy the requrements
00022bd050--6376960256a5a54ced731f40 ACC

00022bd050--637698d656a5a54ced7399ce REJECTED doesn't satisfy the requrements
00022bd050--637698d847f33832e643d8c3 ACCEPTED good job
00022bd050--637698e1c9e3e243ce8832e7 ACCEPTED good job
00022bd050--637698e6838d535336900270 ACCEPTED good job
00022bd050--637698eaea9fec14e37b8ccf ACCEPTED good job
00022bd050--637698ef8408eb4b99990ba0 ACCEPTED good job
00022bd050--637698f8bdfa5861cdf045e1 ACCEPTED good job
00022bd050--637698f97c35c13555f39ce8 ACCEPTED good job
00022bd050--637698fe8feaf022830f8b1a ACCEPTED good job
00022bd050--63769903f63f4544851cead7 ACCEPTED good job
00022bd050--637699068408eb4b99990c45 ACCEPTED good job
00022bd050--637699098ccdeb4a26d4b815 ACCEPTED good job
00022bd050--6376990a56a5a54ced73a1aa ACCEPTED good job
00022bd050--637699111455e654a91a7c83 ACCEPTED good job
00022bd050--6376991256a5a54ced73a3b6 ACCEPTED good job
00022bd050--63769915441f157a2a441e25 ACCEPTED good job
00022bd050--6376991919c2d72a00a6a62f ACCEPTED good job
00022bd050--6376991b56a5a54ced73a521 ACCEP

00022bd050--63769af7ea9fec14e37bcefb ACCEPTED good job
00022bd050--63769afcbdfa5861cdf0559b ACCEPTED good job
00022bd050--63769b075a4d4b776b1216f5 ACCEPTED good job
00022bd050--63769b0aebd2c7014fd6af23 ACCEPTED good job
00022bd050--63769b0fd3a4cf6101906dc9 ACCEPTED good job
00022bd050--63769b12f63f4544851cff24 ACCEPTED good job
00022bd050--63769b189376542ef77d32ed ACCEPTED good job
00022bd050--63769b196b2d840c16ea2d7b ACCEPTED good job
00022bd050--63769b1aee7d356a0869f4c2 ACCEPTED good job
00022bd050--63769b2414cd5667368bd430 ACCEPTED good job
00022bd050--63769b318408eb4b99991e75 ACCEPTED good job
reject 00022bd050--63769b368cae8c05c8196e2b: doesn't satisfy the requrements
00022bd050--63769b368cae8c05c8196e2b REJECTED doesn't satisfy the requrements
00022bd050--63769b368feaf022830f9afa ACCEPTED good job
00022bd050--63769b39bdfa5861cdf05759 ACCEPTED good job
00022bd050--63769b3e3501693ae6553f96 ACCEPTED good job
00022bd050--63769b4dc9e3e243ce88443c ACCEPTED good job
00022bd050--63769b52

...

00022bd050--63775e9d3501693ae65da236 ACCEPTED good job
00022bd050--63775e9d7545ab02b68736c9 ACCEPTED good job
00022bd050--63775e9e8feaf02283153e13 ACCEPTED good job
00022bd050--63775ea01d6dd41f3b969df0 ACCEPTED good job
00022bd050--63775ea18408eb4b99a04179 ACCEPTED good job
reject 00022bd050--63775ea2441f157a2a4c8e7f: doesn't satisfy the requrements
00022bd050--63775ea2441f157a2a4c8e7f REJECTED doesn't satisfy the requrements
reject 00022bd050--63775ea2659b1069108af4d6: doesn't satisfy the requrements
00022bd050--63775ea2659b1069108af4d6 REJECTED doesn't satisfy the requrements
00022bd050--63775ea2b04cf443264674a6 ACCEPTED good job
00022bd050--63775ea38408eb4b99a04187 ACCEPTED good job
00022bd050--63775ea947f33832e64ae075 ACCEPTED good job
00022bd050--63775ea9b2c0c25adcda8662 ACCEPTED good job
00022bd050--63775ea9ee7d356a087124b5 ACCEPTED good job
00022bd050--63775eaa47042e6e2c5641e1 ACCEPTED good job
00022bd050--63775ead38b93276f2ecec93 ACCEPTED good job
00022bd050--63775eae8feaf02283

00022bd050--63776106d3c878690bd06836 REJECTED doesn't satisfy the requrements
00022bd050--63776106db0e8f36d8e88db6 ACCEPTED good job
00022bd050--6377610e838d535336974491 ACCEPTED good job
00022bd050--6377611e1d6dd41f3b96c4e8 ACCEPTED good job
reject 00022bd050--6377612114cd5667369410bf: doesn't satisfy the requrements
00022bd050--6377612114cd5667369410bf REJECTED doesn't satisfy the requrements
00022bd050--6377612556a5a54ced7a91bb ACCEPTED good job
00022bd050--6377613d3501693ae65dd0eb ACCEPTED good job
00022bd050--63776148659b1069108b1184 ACCEPTED good job
00022bd050--63776149f63f4544852530d4 ACCEPTED good job
00022bd050--6377614d838d535336974853 ACCEPTED good job
00022bd050--637761565a4d4b776b183ae7 ACCEPTED good job
00022bd050--63776159bb79c0321f78d04e ACCEPTED good job
00022bd050--6377616285bacb70b670b36e ACCEPTED good job
00022bd050--63776186b2c0c25adcdaa743 ACCEPTED good job
00022bd050--637761953903154c8bb846b6 ACCEPTED good job
00022bd050--637761b747f33832e64b0446 ACCEPTED good j

00022bd050--637778c1db0e8f36d8e983d5 ACCEPTED good job
00022bd050--637778c4441f157a2a4e4305 ACCEPTED good job
00022bd050--637778c8b04cf443264867c4 ACCEPTED good job
00022bd050--637778c9aabecc6bd1766614 ACCEPTED good job
00022bd050--637778cabdfa5861cdf84283 ACCEPTED good job
00022bd050--637778cc47f33832e64c2ec1 ACCEPTED good job
00022bd050--637778ce9717e270005c6c7b ACCEPTED good job
00022bd050--637778d08cae8c05c821a4c8 ACCEPTED good job
00022bd050--637778d2d23d9327ecf894a3 ACCEPTED good job
00022bd050--637778d33903154c8bb990a5 ACCEPTED good job
00022bd050--637778d6c9e3e243ce8f2a02 ACCEPTED good job
00022bd050--637778d6ee7d356a08728b41 ACCEPTED good job
reject 00022bd050--637778d78cae8c05c821a551: doesn't satisfy the requrements
00022bd050--637778d78cae8c05c821a551 REJECTED doesn't satisfy the requrements
reject 00022bd050--637778e18897471c9a8dbe29: doesn't satisfy the requrements
00022bd050--637778e18897471c9a8dbe29 REJECTED doesn't satisfy the requrements
00022bd050--637778e2db0e8f36d8

00022bd050--637779ca8408eb4b99a19fe9 ACCEPTED good job
00022bd050--637779cc801461320b636580 ACCEPTED good job
00022bd050--637779d0b88cb76558ca45f6 ACCEPTED good job
00022bd050--637779d2838d53533698ff31 ACCEPTED good job
00022bd050--637779d68cae8c05c821b39a ACCEPTED good job
00022bd050--637779d747f33832e64c3b1b ACCEPTED good job
00022bd050--637779d81455e654a9233d37 ACCEPTED good job
00022bd050--637779d88897471c9a8dca6d ACCEPTED good job
00022bd050--637779dadb0e8f36d8e98eda ACCEPTED good job
00022bd050--637779dce3847d75ff836e20 ACCEPTED good job
00022bd050--637779e238b93276f2ee5cfb ACCEPTED good job
00022bd050--637779e36656942853152b26 ACCEPTED good job
00022bd050--637779e38897471c9a8dcb24 ACCEPTED good job
00022bd050--637779eb14cd56673695f216 ACCEPTED good job
00022bd050--637779eb441f157a2a4e51f2 ACCEPTED good job
00022bd050--637779edaabecc6bd176734a ACCEPTED good job
00022bd050--637779eebd8f2378d34ddeb7 ACCEPTED good job
00022bd050--637779eed3c878690bd184c4 ACCEPTED good job
00022bd050

00022bd050--63777a9c8ccdeb4a26db526e ACCEPTED good job
00022bd050--63777a9c8ccdeb4a26db5273 ACCEPTED good job
00022bd050--63777aa5838d535336992129 ACCEPTED good job
00022bd050--63777aa6b88cb76558ca5001 ACCEPTED good job
00022bd050--63777aa6bd8f2378d34de756 ACCEPTED good job
00022bd050--63777aabb2c0c25adcdc328a ACCEPTED good job
00022bd050--63777aac19b2bc661e259ed3 ACCEPTED good job
00022bd050--63777aad8d6def0568def9e4 ACCEPTED good job
00022bd050--63777aadaabecc6bd1767a71 ACCEPTED good job
00022bd050--63777aae8408eb4b99a1acb6 ACCEPTED good job
00022bd050--63777ab0f63f45448526b3d1 ACCEPTED good job
00022bd050--63777ab1b58f5e17ab60c675 ACCEPTED good job
00022bd050--63777ab48feaf022831662b3 ACCEPTED good job
reject 00022bd050--63777ab5b04cf443264884b7: doesn't satisfy the requrements
00022bd050--63777ab5b04cf443264884b7 REJECTED doesn't satisfy the requrements
00022bd050--63777ab6441f157a2a4e5bba ACCEPTED good job
00022bd050--63777ab65a4d4b776b198040 ACCEPTED good job
reject 00022bd050--6

In [330]:
df_agreements.to_csv('batch1.csv', index=False)